# Machine Learning Techniques for Sales Forecasting

## Importing Libraries

In [230]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Importing Datasets & Read all csv files

files available at: https://www.kaggle.com/datasets/ndarshan2797/english-converted-datasets

01. item_categories.csv - 
    item_category_name, 
    item_category_id

02. items.csv - 
    item_name, 
    item_id, 
    category_id

03. sales_train.csv - 
    date, 
    date_block_num, 
    shop_id, 
    item_id, 
    item_price, 
    item_cnt_day

04. shops.csv - 
    shop_name, 
    shop_id

05. test.csv - 
    ID, 
    shop_id, 
    item_id

In [231]:
#importing data

item_categories = pd.read_csv('./data-set/item_categories.csv')
items = pd.read_csv('./data-set/items.csv')
sales_train = pd.read_csv('./data-set/sales_train.csv')
shops = pd.read_csv('./data-set/shops.csv')
test = pd.read_csv('./data-set/test.csv')

In [232]:
#checking the shape of the data
print("Shape of item_categories:", item_categories.shape)
print("Shape of items:", items.shape)
print("Shape of sales_train:", sales_train.shape)
print("Shape of shops:", shops.shape)
print("Shape of test:", test.shape)

Shape of item_categories: (84, 2)
Shape of items: (22170, 3)
Shape of sales_train: (2935849, 6)
Shape of shops: (60, 2)
Shape of test: (214200, 3)


In [233]:
#checking the columns of the data
print("\n\nColumns of item_categories:\n")
print(item_categories.info())

print("-----------------------------------------------------")

print("\n\nColumns of items:\n")
print(items.info())

print("-----------------------------------------------------")

print("\n\nColumns of sales_train:\n")
print(sales_train.info())

print("-----------------------------------------------------")

print("\n\nColumns of shops:\n")
print(shops.info())

print("-----------------------------------------------------")

print("\n\nColumns of test:\n")
print(test.info())




Columns of item_categories:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB
None
-----------------------------------------------------


Columns of items:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   item_name    22170 non-null  object
 1   item_id      22170 non-null  int64 
 2   category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB
None
-----------------------------------------------------


Columns of sales_train:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (

In [234]:
#checking the head of the data

print("\n\nHead of item_categories:\n")
print(item_categories.head())

print("-----------------------------------------------------")

print("\n\nHead of items:\n")
print(items.head())

print("-----------------------------------------------------")

print("\n\nHead of sales_train:\n")
print(sales_train.head())

print("-----------------------------------------------------")

print("\n\nHead of shops:\n")
print(shops.head())

print("-----------------------------------------------------")

print("\n\nHead of test:\n")
print(test.head())



Head of item_categories:

           item_category_name  item_category_id
0  PC - Headsets / Headphones                 0
1           Accessories - PS2                 1
2           Accessories - PS3                 2
3           Accessories - PS4                 3
4           Accessories - PSP                 4
-----------------------------------------------------


Head of items:

                                           item_name  item_id  category_id
0             !! IN THE POWER OF HAPPINESS (PLAST) D        0           40
1  ! ABBYY FineReader 12 Professional Edition Ful...        1           76
2              *** IN THE GLORY OF THE GLORY (UNV) D        2           40
3                             *** BLUE WAVE (Univ) D        3           40
4                                  *** BOX (GLASS) D        4           40
-----------------------------------------------------


Head of sales_train:

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2

## Data Preprocessing

In [235]:
#merging the data

In [236]:
#Merge sales_train.csv with items.csv on the "item_id" column
sales_with_items = sales_train.merge(items, on='item_id', how='left')
print("\n\nHead of sales_with_items:\n")
print(sales_with_items.head(20))
print(sales_with_items.shape)



Head of sales_with_items:

          date  date_block_num  shop_id  item_id  item_price  item_cnt_day  \
0   02.01.2013               0       59    22154      999.00           1.0   
1   03.01.2013               0       25     2552      899.00           1.0   
2   05.01.2013               0       25     2552      899.00          -1.0   
3   06.01.2013               0       25     2554     1709.05           1.0   
4   15.01.2013               0       25     2555     1099.00           1.0   
5   10.01.2013               0       25     2564      349.00           1.0   
6   02.01.2013               0       25     2565      549.00           1.0   
7   04.01.2013               0       25     2572      239.00           1.0   
8   11.01.2013               0       25     2572      299.00           1.0   
9   03.01.2013               0       25     2573      299.00           3.0   
10  03.01.2013               0       25     2574      399.00           2.0   
11  05.01.2013               0     

In [237]:
#Merge the result with item_categories.csv on the "category_id" 
sales_with_items_and_categories = sales_with_items.merge(item_categories, right_on='item_category_id', left_on='category_id', how='left')
print("\n\nHead of sales_with_items_and_categories:\n")
print(sales_with_items_and_categories.head(20))
print(sales_with_items_and_categories.shape)



Head of sales_with_items_and_categories:

          date  date_block_num  shop_id  item_id  item_price  item_cnt_day  \
0   02.01.2013               0       59    22154      999.00           1.0   
1   03.01.2013               0       25     2552      899.00           1.0   
2   05.01.2013               0       25     2552      899.00          -1.0   
3   06.01.2013               0       25     2554     1709.05           1.0   
4   15.01.2013               0       25     2555     1099.00           1.0   
5   10.01.2013               0       25     2564      349.00           1.0   
6   02.01.2013               0       25     2565      549.00           1.0   
7   04.01.2013               0       25     2572      239.00           1.0   
8   11.01.2013               0       25     2572      299.00           1.0   
9   03.01.2013               0       25     2573      299.00           3.0   
10  03.01.2013               0       25     2574      399.00           2.0   
11  05.01.2013      

In [238]:
# Check if the two columns are the same
if sales_with_items_and_categories['item_category_id'].equals(sales_with_items_and_categories['category_id']):
    # If they are the same, you can drop one of the columns
    sales_with_items_and_categories.drop(columns=['item_category_id'], inplace=True)


In [239]:
print("\n\nHead of sales_with_items_and_categories:\n")
print(sales_with_items_and_categories.head(20))
print(sales_with_items_and_categories.shape)



Head of sales_with_items_and_categories:

          date  date_block_num  shop_id  item_id  item_price  item_cnt_day  \
0   02.01.2013               0       59    22154      999.00           1.0   
1   03.01.2013               0       25     2552      899.00           1.0   
2   05.01.2013               0       25     2552      899.00          -1.0   
3   06.01.2013               0       25     2554     1709.05           1.0   
4   15.01.2013               0       25     2555     1099.00           1.0   
5   10.01.2013               0       25     2564      349.00           1.0   
6   02.01.2013               0       25     2565      549.00           1.0   
7   04.01.2013               0       25     2572      239.00           1.0   
8   11.01.2013               0       25     2572      299.00           1.0   
9   03.01.2013               0       25     2573      299.00           3.0   
10  03.01.2013               0       25     2574      399.00           2.0   
11  05.01.2013      

In [240]:
#Merge the result with shops.csv on the "shop_id" 
final_dataset = sales_with_items_and_categories.merge(shops, on='shop_id', how='left')
print("\n\nHead of final_dataset:\n")
print(final_dataset.head(20))
print(final_dataset.shape)



Head of final_dataset:

          date  date_block_num  shop_id  item_id  item_price  item_cnt_day  \
0   02.01.2013               0       59    22154      999.00           1.0   
1   03.01.2013               0       25     2552      899.00           1.0   
2   05.01.2013               0       25     2552      899.00          -1.0   
3   06.01.2013               0       25     2554     1709.05           1.0   
4   15.01.2013               0       25     2555     1099.00           1.0   
5   10.01.2013               0       25     2564      349.00           1.0   
6   02.01.2013               0       25     2565      549.00           1.0   
7   04.01.2013               0       25     2572      239.00           1.0   
8   11.01.2013               0       25     2572      299.00           1.0   
9   03.01.2013               0       25     2573      299.00           3.0   
10  03.01.2013               0       25     2574      399.00           2.0   
11  05.01.2013               0       2

In [241]:
#checks the columns of the final dataset
print("\n\nColumns of final_dataset:\n")
print(final_dataset.info())




Columns of final_dataset:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                object 
 1   date_block_num      int64  
 2   shop_id             int64  
 3   item_id             int64  
 4   item_price          float64
 5   item_cnt_day        float64
 6   item_name           object 
 7   category_id         int64  
 8   item_category_name  object 
 9   shop_name           object 
dtypes: float64(2), int64(4), object(4)
memory usage: 224.0+ MB
None


In [242]:
#prints the date and date_block_num column to check whether they are related
columns_to_print = ['date', 'date_block_num']
print(final_dataset[columns_to_print])

               date  date_block_num
0        02.01.2013               0
1        03.01.2013               0
2        05.01.2013               0
3        06.01.2013               0
4        15.01.2013               0
...             ...             ...
2935844  10.10.2015              33
2935845  09.10.2015              33
2935846  14.10.2015              33
2935847  22.10.2015              33
2935848  03.10.2015              33

[2935849 rows x 2 columns]


In [243]:
# Rename the column
final_dataset.rename(columns={'date_block_num': 'month_num'}, inplace=True)

In [244]:
#Rename the item_cnt_day column
final_dataset.rename(columns={'item_cnt_day': 'item_cnt_month'}, inplace=True)

In [245]:
print("\n\nHead of final_dataset:\n")
print(final_dataset.head(20))
print(final_dataset.shape)



Head of final_dataset:

          date  month_num  shop_id  item_id  item_price  item_cnt_month  \
0   02.01.2013          0       59    22154      999.00             1.0   
1   03.01.2013          0       25     2552      899.00             1.0   
2   05.01.2013          0       25     2552      899.00            -1.0   
3   06.01.2013          0       25     2554     1709.05             1.0   
4   15.01.2013          0       25     2555     1099.00             1.0   
5   10.01.2013          0       25     2564      349.00             1.0   
6   02.01.2013          0       25     2565      549.00             1.0   
7   04.01.2013          0       25     2572      239.00             1.0   
8   11.01.2013          0       25     2572      299.00             1.0   
9   03.01.2013          0       25     2573      299.00             3.0   
10  03.01.2013          0       25     2574      399.00             2.0   
11  05.01.2013          0       25     2574      399.00             1.0   

In [246]:
#checks the columns of the final dataset
print("\n\nColumns of final_dataset:\n")
print(final_dataset.info())



Columns of final_dataset:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                object 
 1   month_num           int64  
 2   shop_id             int64  
 3   item_id             int64  
 4   item_price          float64
 5   item_cnt_month      float64
 6   item_name           object 
 7   category_id         int64  
 8   item_category_name  object 
 9   shop_name           object 
dtypes: float64(2), int64(4), object(4)
memory usage: 224.0+ MB
None


In [247]:
#Data Cleaning

#checking for missing values
print("\n\nMissing values in final_dataset:\n")
print(final_dataset.isnull().sum())



Missing values in final_dataset:



date                  0
month_num             0
shop_id               0
item_id               0
item_price            0
item_cnt_month        0
item_name             0
category_id           0
item_category_name    0
shop_name             0
dtype: int64


In [248]:
#checking for null values
print("\n\nNull values in final_dataset:\n")
print(final_dataset.isnull().sum())



Null values in final_dataset:

date                  0
month_num             0
shop_id               0
item_id               0
item_price            0
item_cnt_month        0
item_name             0
category_id           0
item_category_name    0
shop_name             0
dtype: int64


In [249]:
print(final_dataset.shape)

(2935849, 10)


In [250]:
#handles the missing values in final_dataset
final_dataset['item_name'].fillna('Unknown', inplace=True)
final_dataset['item_category_name'].fillna('Unknown', inplace=True)

In [251]:
print(final_dataset.shape)

(2935849, 10)


In [252]:
#removes duplicates rows in final_dataset
final_dataset.drop_duplicates(inplace=True)


In [253]:
print(final_dataset.shape)

(2935843, 10)


In [254]:
#checks and solves the data type of the columns
print("\n\nData types of final_dataset:\n")
print(final_dataset.dtypes)




Data types of final_dataset:

date                   object
month_num               int64
shop_id                 int64
item_id                 int64
item_price            float64
item_cnt_month        float64
item_name              object
category_id             int64
item_category_name     object
shop_name              object
dtype: object


In [255]:
#seems like item_cnt_month should be int64
final_dataset['item_cnt_month'] = final_dataset['item_cnt_month'].astype('int64')

In [256]:
print(final_dataset.dtypes)

date                   object
month_num               int64
shop_id                 int64
item_id                 int64
item_price            float64
item_cnt_month          int64
item_name              object
category_id             int64
item_category_name     object
shop_name              object
dtype: object


In [257]:
#prints item_cnt_month column to check whether it is int64
print(final_dataset['item_cnt_month'].head(30))   

0     1
1     1
2    -1
3     1
4     1
5     1
6     1
7     1
8     1
9     3
10    2
11    1
12    1
13    2
14    1
15    2
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
Name: item_cnt_month, dtype: int64


In [258]:
print(final_dataset.shape)

(2935843, 10)


In [259]:
#removes -1 and 307980 from item_cnt_month column
#because it is an outlier
#and it is not possible to sell -1 and 307980 items in a day
#because 307980 is the total number of items sold in a day
#which means that the data is incorrect
#and -1 is not possible
#which means that the data is incorrect

final_dataset = final_dataset[(final_dataset['item_cnt_month'] > 0) & (final_dataset['item_cnt_month'] < 307980)]

print(final_dataset.shape)

(2928487, 10)


In [260]:
#deal with the incorrect data in the item_price column
#the item_price should not be negative
#the item_price should not be zero
#the item_price should not be greater than 100000

final_dataset = final_dataset[(final_dataset['item_price'] > 0) & (final_dataset['item_price'] < 100000)]

In [261]:
print(final_dataset.shape)

(2928485, 10)


In [262]:
#handles special characters and formatting in the data set
final_dataset['item_name'] = final_dataset['item_name'].str.replace('[^A-Za-z0-9А-Яа-я]+', ' ')

In [263]:
print(final_dataset.shape)

(2928485, 10)


In [264]:
#handles noise in the data set

#removes the noise in the item_name column
final_dataset['item_name'] = final_dataset['item_name'].str.replace('  ', ' ')

In [265]:
print(final_dataset.head())

         date  month_num  shop_id  item_id  item_price  item_cnt_month  \
0  02.01.2013          0       59    22154      999.00               1   
1  03.01.2013          0       25     2552      899.00               1   
3  06.01.2013          0       25     2554     1709.05               1   
4  15.01.2013          0       25     2555     1099.00               1   
5  10.01.2013          0       25     2564      349.00               1   

                                           item_name  category_id  \
0                                    SCENE 2012 (BD)           37   
1             DEEP PURPLE The House Of Blue Light LP           58   
3             DEEP PURPLE Who Do You Think We Are LP           58   
4      DEEP PURPLE 30 Very Best Of 2CD (Businesses).           56   
5  DEEP PURPLE Perihelion: Live In Concert DVD (C...           59   

                    item_category_name                shop_name  
0                     Cinema - Blu-Ray   Yaroslavl TC" Altair "  
1       

In [266]:
#data integration

#creates a new column called revenue
final_dataset['revenue'] = final_dataset['item_cnt_month'] * final_dataset['item_price']


In [267]:
print("\n\nHead of final_dataset:\n")
print(final_dataset.head(20))
print(final_dataset.shape)



Head of final_dataset:

          date  month_num  shop_id  item_id  item_price  item_cnt_month  \
0   02.01.2013          0       59    22154      999.00               1   
1   03.01.2013          0       25     2552      899.00               1   
3   06.01.2013          0       25     2554     1709.05               1   
4   15.01.2013          0       25     2555     1099.00               1   
5   10.01.2013          0       25     2564      349.00               1   
6   02.01.2013          0       25     2565      549.00               1   
7   04.01.2013          0       25     2572      239.00               1   
8   11.01.2013          0       25     2572      299.00               1   
9   03.01.2013          0       25     2573      299.00               3   
10  03.01.2013          0       25     2574      399.00               2   
11  05.01.2013          0       25     2574      399.00               1   
12  07.01.2013          0       25     2574      399.00               1   

In [268]:
#data integration and data transformation

#creates a new column called date num
final_dataset['date_num'] = final_dataset['date'].str[:2]

In [269]:
print("\n\nHead of final_dataset:\n")
print(final_dataset.head(20))
print(final_dataset.shape)



Head of final_dataset:

          date  month_num  shop_id  item_id  item_price  item_cnt_month  \
0   02.01.2013          0       59    22154      999.00               1   
1   03.01.2013          0       25     2552      899.00               1   
3   06.01.2013          0       25     2554     1709.05               1   
4   15.01.2013          0       25     2555     1099.00               1   
5   10.01.2013          0       25     2564      349.00               1   
6   02.01.2013          0       25     2565      549.00               1   
7   04.01.2013          0       25     2572      239.00               1   
8   11.01.2013          0       25     2572      299.00               1   
9   03.01.2013          0       25     2573      299.00               3   
10  03.01.2013          0       25     2574      399.00               2   
11  05.01.2013          0       25     2574      399.00               1   
12  07.01.2013          0       25     2574      399.00               1   

In [270]:
#data integration

#creates a new column called year num
final_dataset['year_num'] = final_dataset['date'].str[6:]

In [271]:
print("\n\nHead of final_dataset:\n")
print(final_dataset.head(20))
print(final_dataset.shape)



Head of final_dataset:

          date  month_num  shop_id  item_id  item_price  item_cnt_month  \
0   02.01.2013          0       59    22154      999.00               1   
1   03.01.2013          0       25     2552      899.00               1   
3   06.01.2013          0       25     2554     1709.05               1   
4   15.01.2013          0       25     2555     1099.00               1   
5   10.01.2013          0       25     2564      349.00               1   
6   02.01.2013          0       25     2565      549.00               1   
7   04.01.2013          0       25     2572      239.00               1   
8   11.01.2013          0       25     2572      299.00               1   
9   03.01.2013          0       25     2573      299.00               3   
10  03.01.2013          0       25     2574      399.00               2   
11  05.01.2013          0       25     2574      399.00               1   
12  07.01.2013          0       25     2574      399.00               1   

In [272]:
print(final_dataset.shape)
print(final_dataset.info())

(2928485, 13)
<class 'pandas.core.frame.DataFrame'>
Index: 2928485 entries, 0 to 2935848
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                object 
 1   month_num           int64  
 2   shop_id             int64  
 3   item_id             int64  
 4   item_price          float64
 5   item_cnt_month      int64  
 6   item_name           object 
 7   category_id         int64  
 8   item_category_name  object 
 9   shop_name           object 
 10  revenue             float64
 11  date_num            object 
 12  year_num            object 
dtypes: float64(2), int64(5), object(6)
memory usage: 312.8+ MB
None


In [273]:
# rearrange the columns
final_dataset = final_dataset[['date', 'date_num', 'year_num', 'month_num', 'shop_id', 'shop_name', 'item_id', 'item_name', 'category_id', 'item_category_name', 'item_price', 'item_cnt_month', 'revenue']]

print(final_dataset.shape)
print(final_dataset.info())

(2928485, 13)
<class 'pandas.core.frame.DataFrame'>
Index: 2928485 entries, 0 to 2935848
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                object 
 1   date_num            object 
 2   year_num            object 
 3   month_num           int64  
 4   shop_id             int64  
 5   shop_name           object 
 6   item_id             int64  
 7   item_name           object 
 8   category_id         int64  
 9   item_category_name  object 
 10  item_price          float64
 11  item_cnt_month      int64  
 12  revenue             float64
dtypes: float64(2), int64(5), object(6)
memory usage: 312.8+ MB
None


In [274]:
#since we already handled the missing and null values in the data set there is no need for imputation

In [275]:
#data profiling

#descriptive statistics
print("\n\nDescriptive statistics of final_dataset:\n")
print(final_dataset.describe())



Descriptive statistics of final_dataset:

          month_num       shop_id       item_id   category_id    item_price  \
count  2.928485e+06  2.928485e+06  2.928485e+06  2.928485e+06  2.928485e+06   
mean   1.456977e+01  3.300294e+01  1.020029e+04  4.001637e+01  8.893621e+02   
std    9.422957e+00  1.622543e+01  6.324392e+03  1.709811e+01  1.718155e+03   
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  7.000000e-02   
25%    7.000000e+00  2.200000e+01  4.477000e+03  2.800000e+01  2.490000e+02   
50%    1.400000e+01  3.100000e+01  9.355000e+03  4.000000e+01  3.990000e+02   
75%    2.300000e+01  4.700000e+01  1.569100e+04  5.500000e+01  9.990000e+02   
max    3.300000e+01  5.900000e+01  2.216900e+04  8.300000e+01  5.920000e+04   

       item_cnt_month       revenue  
count    2.928485e+06  2.928485e+06  
mean     1.248338e+00  1.164269e+03  
std      2.619589e+00  5.684852e+03  
min      1.000000e+00  7.000000e-02  
25%      1.000000e+00  2.490000e+02  
50%      1.00000

In [276]:
#data enrichment

#creates a new column called month name
final_dataset['month_name'] = final_dataset['month_num'].replace({0: 'January', 1: 'February', 2: 'March', 3: 'April', 4: 'May', 5: 'June', 6: 'July', 7: 'August', 8: 'September', 9: 'October', 10: 'November', 11: 'December'})

print("\n\nHead of final_dataset:\n")
print(final_dataset.head(20))
print(final_dataset.shape)



Head of final_dataset:

          date date_num year_num  month_num  shop_id                shop_name  \
0   02.01.2013       02     2013          0       59   Yaroslavl TC" Altair "   
1   03.01.2013       03     2013          0       25     Moscow TEC" Atrium "   
3   06.01.2013       06     2013          0       25     Moscow TEC" Atrium "   
4   15.01.2013       15     2013          0       25     Moscow TEC" Atrium "   
5   10.01.2013       10     2013          0       25     Moscow TEC" Atrium "   
6   02.01.2013       02     2013          0       25     Moscow TEC" Atrium "   
7   04.01.2013       04     2013          0       25     Moscow TEC" Atrium "   
8   11.01.2013       11     2013          0       25     Moscow TEC" Atrium "   
9   03.01.2013       03     2013          0       25     Moscow TEC" Atrium "   
10  03.01.2013       03     2013          0       25     Moscow TEC" Atrium "   
11  05.01.2013       05     2013          0       25     Moscow TEC" Atrium "   
12

In [277]:
#removes month_num column

final_dataset.drop(columns=['month_num'], inplace=True)

print("\n\nHead of final_dataset:\n")
print(final_dataset.head(20))
print(final_dataset.shape)



Head of final_dataset:

          date date_num year_num  shop_id                shop_name  item_id  \
0   02.01.2013       02     2013       59   Yaroslavl TC" Altair "    22154   
1   03.01.2013       03     2013       25     Moscow TEC" Atrium "     2552   
3   06.01.2013       06     2013       25     Moscow TEC" Atrium "     2554   
4   15.01.2013       15     2013       25     Moscow TEC" Atrium "     2555   
5   10.01.2013       10     2013       25     Moscow TEC" Atrium "     2564   
6   02.01.2013       02     2013       25     Moscow TEC" Atrium "     2565   
7   04.01.2013       04     2013       25     Moscow TEC" Atrium "     2572   
8   11.01.2013       11     2013       25     Moscow TEC" Atrium "     2572   
9   03.01.2013       03     2013       25     Moscow TEC" Atrium "     2573   
10  03.01.2013       03     2013       25     Moscow TEC" Atrium "     2574   
11  05.01.2013       05     2013       25     Moscow TEC" Atrium "     2574   
12  07.01.2013       07   

In [278]:
#rearange the columns

final_dataset = final_dataset[['date', 'date_num', 'month_name', 'year_num', 'shop_id', 'shop_name', 'item_id', 'item_name', 'category_id', 'item_category_name', 'item_price', 'item_cnt_month', 'revenue']]

print("\n\nHead of final_dataset:\n")
print(final_dataset.head(20))
print(final_dataset.shape)



Head of final_dataset:

          date date_num month_name year_num  shop_id                shop_name  \
0   02.01.2013       02    January     2013       59   Yaroslavl TC" Altair "   
1   03.01.2013       03    January     2013       25     Moscow TEC" Atrium "   
3   06.01.2013       06    January     2013       25     Moscow TEC" Atrium "   
4   15.01.2013       15    January     2013       25     Moscow TEC" Atrium "   
5   10.01.2013       10    January     2013       25     Moscow TEC" Atrium "   
6   02.01.2013       02    January     2013       25     Moscow TEC" Atrium "   
7   04.01.2013       04    January     2013       25     Moscow TEC" Atrium "   
8   11.01.2013       11    January     2013       25     Moscow TEC" Atrium "   
9   03.01.2013       03    January     2013       25     Moscow TEC" Atrium "   
10  03.01.2013       03    January     2013       25     Moscow TEC" Atrium "   
11  05.01.2013       05    January     2013       25     Moscow TEC" Atrium "   
12

## Feature Engineering